# Experiments
The goal is to experiment with different prompts to determine which features provide the most value to the overall experiment. To achieve this, we will test various techniques until we identify the most effective one, which will then be selected for deployment.

# Testing

In [1]:
import openai
from openai import OpenAI
import json
import re
import helper.creden as cre
from helper.helper import parse_conversation, get_completion_from_messages, select_random_test

# widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# import variables
# openai.api_key = cre.OPENAI_API_KEY
client = OpenAI(api_key=cre.OPENAI_API_KEY)
model_selection="gpt-4o-mini"

In [2]:
response = client.chat.completions.create(
    model=model_selection,
    messages=[{"role": "user", "content": "hola como estas"}],
    max_tokens=64
)
response

ChatCompletion(id='chatcmpl-AvMwHh4dW8B6t3LmnA7NjmmB5u4Hs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='¡Hola! Estoy aquí y listo para ayudarte. ¿En qué puedo asistirte hoy?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738236237, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_72ed7ab54c', usage=CompletionUsage(completion_tokens=19, prompt_tokens=10, total_tokens=29, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [3]:
print(response.choices[0].message.role, " :")
print(response.choices[0].message.content)
print(response.usage)

print(response.usage.prompt_tokens)
print(response.usage.completion_tokens)
print(response.usage.total_tokens)

assistant  :
¡Hola! Estoy aquí y listo para ayudarte. ¿En qué puedo asistirte hoy?
CompletionUsage(completion_tokens=19, prompt_tokens=10, total_tokens=29, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
10
19
29


Now that we have confirmed the API is working, we will conduct several experiments to determine which prompts provide the most value. Next, we will select a theme for the interview to ensure a structured approach.

In [4]:
tests_interviews = {
                "topics": [
                    {
                    "name": "Predictive Modeling",
                    "example": "Predicting customer churn for a subscription service.",
                    "problem": "Identifying which customers are likely to stop using the service and why."
                    },
                    {
                    "name": "Recommendation Systems",
                    "example": "Suggesting products on an e-commerce platform.",
                    "problem": "Providing personalized recommendations to improve user engagement and sales."
                    },
                    {
                    "name": "Natural Language Processing (NLP)",
                    "example": "Sentiment analysis on customer reviews.",
                    "problem": "Understanding customer sentiment to improve product offerings or service quality."
                    },
                    {
                    "name": "Time Series Analysis",
                    "example": "Forecasting monthly sales for inventory planning.",
                    "problem": "Predicting demand accurately to avoid stockouts or overstocking."
                    },
                    {
                    "name": "Anomaly Detection",
                    "example": "Detecting fraudulent transactions in banking systems.",
                    "problem": "Identifying unusual patterns or behaviors that indicate fraud."
                    },
                    {
                    "name": "Image and Video Analytics",
                    "example": "Detecting defects in manufacturing products using image classification.",
                    "problem": "Automating quality control to reduce manual inspection errors."
                    },
                    {
                    "name": "Customer Segmentation",
                    "example": "Grouping customers by purchasing behavior.",
                    "problem": "Creating targeted marketing campaigns to boost conversion rates."
                    },
                    {
                    "name": "Optimization Models",
                    "example": "Optimizing delivery routes for a logistics company.",
                    "problem": "Minimizing fuel costs and delivery time."
                    },
                    {
                    "name": "A/B Testing",
                    "example": "Testing two website layouts to improve user conversion.",
                    "problem": "Determining which design leads to better engagement or sales."
                    },
                    {
                    "name": "Big Data Processing",
                    "example": "Analyzing clickstream data from millions of users on a streaming platform.",
                    "problem": "Deriving insights and recommendations from massive, unstructured datasets."
                    }
                ]
                }
# select a random test
random_topic = select_random_test(tests_interviews)
random_topic

{'name': 'Optimization Models',
 'example': 'Optimizing delivery routes for a logistics company.',
 'problem': 'Minimizing fuel costs and delivery time.'}

# Experiments

### Zero-Shot Prompting

In this case, we will simply structure the interview prompts as we see fit.

In [14]:
panels = [] # collect display 

prompt_example = f"""
We are making a datascience interview to a person, we need to just talk about that topic, we are not allowed to talk about anything else.
Try to continue to the next question once the answer its done.

You select the 3 question to ask about data science, but has to be related to this topic  {random_topic["example"]} and about this problem {random_topic["problem"]} , add 2 points for each correct question and use discretion to evaluate it.

After the question replay what you do good in the past question but dont give an answer of it.

You can make a json with the result in points for each question if there are correct 2 if not 0 question.

Once you put the json you can ask if there its anything else, if not say good bye
"""

context = [{'role':'system', 'content':prompt_example}]
context

[{'role': 'system',
  'content': '\nWe are making a datascience interview to a person, we need to just talk about that topic, we are not allowed to talk about anything else.\nTry to continue to the next question once the answer its done.\n\nYou select the 3 question to ask about data science, but has to be related to this topic  Optimizing delivery routes for a logistics company. and about this problem Minimizing fuel costs and delivery time. , add 2 points for each correct question and use discretion to evaluate it.\n\nAfter the question replay what you do good in the past question but dont give an answer of it.\n\nYou can make a json with the result in points for each question if there are correct 2 if not 0 question.\n\nOnce you put the json you can ask if there its anything else, if not say good bye\n'}]

In [16]:
# Create text box for user input
user_input = widgets.Textarea(value='',
                              placeholder='Enter text here...',
                              description='User:',
                              rows=2,
                              layout=widgets.Layout(width='auto'))

# Create text area for conversation display
conversation_history = widgets.Textarea(value='',
                                        description='Chat:',
                                        rows=15,
                                        layout=widgets.Layout(width='auto'))

# Create button for user to click to send their message
send_button = widgets.Button(description='Send')

# Initial Question (without response)
initial_question = "We will start with the interview. Can you give me a brief introduction about yourself?"
conversation_history.value += 'assistant: ' + initial_question + '\n'
context.append({'role': 'assistant', 'content': initial_question})

# -------------------------------
def send_button_clicked(b):
    """Send content to the chat"""
    global context
    
    conversation_history.value += 'user: ' + user_input.value + '\n'
    context.append({'role': 'user', 'content': user_input.value})
    response = get_completion_from_messages(context, model = model_selection, temperature=0.8, max_tokens_values=80) 
    conversation_history.value += 'assistant: ' + response + '\n'
    context.append({'role': 'assistant', 'content': response})
    user_input.value = ''
    
# -------------------------------
send_button.on_click(send_button_clicked)

# -------------------------------
# Add keyboard shortcut for sending (Shift+Enter)
def handle_keypress(widget):
    if widget.value.endswith('\n'):
        send_button_clicked(None)
        
user_input.observe(handle_keypress, names='value')
# -------------------------------

# Display everything
display(conversation_history, user_input, send_button)

Textarea(value='assistant: We will start with the interview. Can you give me a brief introduction about yourse…

Textarea(value='', description='User:', layout=Layout(width='auto'), placeholder='Enter text here...', rows=2)

Button(description='Send', style=ButtonStyle())

In [20]:
json_result = parse_conversation(conversation_history.value)
print(json.dumps(json_result, indent=2))

{
  "conversation": [
    {
      "role": "assistant",
      "content": "We will start with the interview. Can you give me a brief introduction about yourself?"
    },
    {
      "role": "user",
      "content": "im kaly"
    },
    {
      "role": "assistant",
      "content": "Thank you, Kaly. Let's dive into the questions related to optimizing delivery routes for a logistics company.\n**Question 1:** What data sources would you consider essential for optimizing delivery routes, and how would you utilize them to minimize fuel costs and delivery time? \n(Please provide your answer.)"
    },
    {
      "role": "user",
      "content": "using knn as a source for finding the best routes"
    },
    {
      "role": "assistant",
      "content": "Thank you for your response. Using KNN (K-Nearest Neighbors) can be a useful approach for finding optimal routes based on historical data. However, there are other factors and data sources that could also be considered for a more comprehensive s

In [24]:
pattern = r'"results":\s*{(.*?)}'

match = re.search(pattern, conversation_history.value, re.DOTALL)

if match:
    extracted_data = match.group(1)
    print(f"Extracted Data:\n{extracted_data}")
else:
    print("No match found.")

Extracted Data:

    "question_1": 0,
    "question_2": 2,
    "question_3": 0
  


As we can see from this test, we didn’t pass it, so the experiment is being qualified correctly.
However, I believe we could improve the model’s performance,
and we'll try using other techniques to see how it performs.
also we could see that some questions aren’t focused on a proper interview, so i guess its not the best method to approach this task.